# 1. Project Overview & Business Understanding



The head of a new movie studio requires data-driven recommendations to guide initial film production choices, specifically aiming to maximize worldwide box office success.



Key Questions to be Answered:



1. Which film genres yield the highest average worldwide revenue?


2. How does audience reception (IMDB rating) correlate with financial success?


3. Is there an optimal film runtime that maximizes gross earnings?


4. How consistent are audience ratings within the highest-grossing film genres?


5. Are the observed differences in average genre revenue statistically significant, or merely due to random chance?

2. Data Understanding and Acquisition

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import sqlite3

In [2]:
# Establish connection to the IMDB database
conn = sqlite3.connect('im.db')
# Load the Box Office Mojo (BOM) gross revenue data
df = pd.read_csv('bom.movie_gross.csv.gz')

In [3]:
# Display all tables in the database
query1 = """
SELECT * 
FROM sqlite_master
 WHERE type = 'table';
 """
print(pd.read_sql_query(query1, conn))

    type           name       tbl_name  rootpage  \
0  table   movie_basics   movie_basics         2   
1  table      directors      directors         3   
2  table      known_for      known_for         4   
3  table     movie_akas     movie_akas         5   
4  table  movie_ratings  movie_ratings         6   
5  table        persons        persons         7   
6  table     principals     principals         8   
7  table        writers        writers         9   
8  table        Revenue        Revenue     41369   

                                                 sql  
0  CREATE TABLE "movie_basics" (\n"movie_id" TEXT...  
1  CREATE TABLE "directors" (\n"movie_id" TEXT,\n...  
2  CREATE TABLE "known_for" (\n"person_id" TEXT,\...  
3  CREATE TABLE "movie_akas" (\n"movie_id" TEXT,\...  
4  CREATE TABLE "movie_ratings" (\n"movie_id" TEX...  
5  CREATE TABLE "persons" (\n"person_id" TEXT,\n ...  
6  CREATE TABLE "principals" (\n"movie_id" TEXT,\...  
7  CREATE TABLE "writers" (\n"movie_id"

In [4]:
print("\nBox Office Mojo Data Preview:")
# Preview BOM data structure
print(df.head())


Box Office Mojo Data Preview:
                                         title studio  domestic_gross  \
0                                  Toy Story 3     BV     415000000.0   
1                   Alice in Wonderland (2010)     BV     334200000.0   
2  Harry Potter and the Deathly Hallows Part 1     WB     296000000.0   
3                                    Inception     WB     292600000.0   
4                          Shrek Forever After   P/DW     238700000.0   

  foreign_gross  year  
0     652000000  2010  
1     691300000  2010  
2     664300000  2010  
3     535700000  2010  
4     513900000  2010  
